# All CMIP6 Phase 2 GCM 2070-2100

Notebook environment to migrate NetCDF files to CF compliant zarr

In [1]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Import standard packages
import os
import pathlib
import sys

import numpy as np
#import geopandas as gpd
 
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import glob
import itertools
import json
import copy
from itertools import chain
from shapely import wkb
import dask
from dask.diagnostics import ProgressBar
from tqdm import tqdm

# Import custom functionality
# sys.path.append('c:/Windows/System32/coclicodata/src')#CLENMAR ADDED

from coclicodata.drive_config import p_drive
from coclicodata.etl.cf_compliancy_checker import check_compliancy, save_compliancy
#P:\11209197-018-global-coastal-atlas\MSc_students\ClenmarRowe\Data\All_Datasets\Orig_Datasets\01_Hazards\01_Present\Extreme_Wave_Height
# Define (local and) remote drives
# gca_data_dir = pathlib.Path.home().joinpath(r"OneDrive - Stichting Deltares\Documents\GitHub")
gca_data_dir = p_drive.joinpath("11209197-018-global-coastal-atlas", r"MSc_students\ClenmarRowe\Data\All_Datasets", r"Orig_Datasets\01_Hazards\02_Future")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    pathlib.Path().home().joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"AppData\Local\miniconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml" ###Changed from anaconda to miniconda  - added the new package udunits file
    )
)


<IPython.core.display.Javascript object>

In [3]:
GCM_all=["MRI-ESM2-0","MPI-ESM1-2-LR","KIOST-ESM","IPSL-CM6A-LR","CMCC-CM2-SR5","AWI-CM-1-1-MR"]

<IPython.core.display.Javascript object>

In [4]:
for i,GCM in tqdm(enumerate(GCM_all),desc="GCM completed"):# Project paths & files (manual input)

    with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    
        dataset_dir = gca_data_dir.joinpath(r"Extreme_Wave_Height\GCM_All_CMIP6-phase2_Projection_2071-2100",GCM)
        # os.mkdir(dataset_dir)
        dataset_dir_path = gca_data_dir.joinpath(r"Extreme_Wave_Height\GCM_All_CMIP6-phase2_Projection_2071-2100",GCM,GCM+"_WW3_3Hourly_2071-2100.nc")
        CF_dir = gca_data_dir.joinpath(r"Extreme_Wave_Height\GCM_All_CMIP6-phase2_Projection_2071-2100",GCM,"CF")  # directory to save output CF check files
        
        
        #For SSP1-26

        orig_dataset_dir= gca_data_dir.joinpath(r"Extreme_Wave_Height\GCM_All_CMIP6-phase2_Projection_2071-2100")
        # Specify the path to the directory containing the netCDF files
    
        ds=xr.open_zarr(str(dataset_dir_path).replace(".nc", ".zarr"))
        ds = ds.chunk({'time': 1, 'latitude': 361, 'longitude': 720})  
        

        if i == 0:
            combined_all = ds
        else:
            combined_all['hs_5_85_' + GCM.replace("-", "_")] = ds['hs_5_85_' + GCM.replace("-", "_")]
            combined_all['hs_1_26_' + GCM.replace("-", "_")] = ds['hs_1_26_' + GCM.replace("-", "_")]
combined_all
    
# # export to zarr in write mode (to overwrite if exists)
# ds.to_zarr(str(dataset_dir_path).replace(".nc", ".zarr"), mode="w")

GCM completed: 6it [00:12,  2.12s/it]


<xarray.Dataset>
Dimensions:                (latitude: 361, longitude: 720, time: 87656)
Coordinates:
  * latitude               (latitude) float32 -90.0 -89.5 -89.0 ... 89.5 90.0
  * longitude              (longitude) float32 0.0 0.5 1.0 ... 358.5 359.0 359.5
  * time                   (time) datetime64[ns] 2071-01-01 ... 2100-12-31T21...
Data variables:
    hs_1_26_MRI_ESM2_0     (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_MRI_ESM2_0     (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_MPI_ESM1_2_LR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_MPI_ESM1_2_LR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_KIOST_ESM      (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_KIOST_ESM      (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_IPSL_CM6A_LR   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_IPSL_CM6A_LR   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_CMCC_CM2_SR5   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_CMCC_CM2_SR5   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_AWI_CM_1_1_MR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_AWI_CM_1_1_MR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
Attributes: (12/36)
    AUTHOR:                        Commonwealth Scientific and Industrial Res...
    CITATION:                      CSIRO. (Year).  2071-2100 CMIP6 MRI-ESM2-0...
    COMMENT:                       The dataset includes significant wave heig...
    CRS:                           EPSG:4326
    Conventions:                   CF-1.8
    DESCRIPTION:                   The dataset contains projections of signif...
    ...                            ...
    northernmost_latitude:         90.
    product_name:                  ww3.207101_hs.nc
    southernmost_latitude:         -90.
    start_date:                    2071-01-01 00:00:00
    stop_date:                     2071-01-31 21:00:00
    westernmost_longitude:         0.

<IPython.core.display.Javascript object>

In [5]:
combined_all_rechunk = combined_all.chunk({'time': 1, 'latitude': 361, 'longitude': 720})   
combined_all_rechunk

<xarray.Dataset>
Dimensions:                (latitude: 361, longitude: 720, time: 87656)
Coordinates:
  * latitude               (latitude) float32 -90.0 -89.5 -89.0 ... 89.5 90.0
  * longitude              (longitude) float32 0.0 0.5 1.0 ... 358.5 359.0 359.5
  * time                   (time) datetime64[ns] 2071-01-01 ... 2100-12-31T21...
Data variables:
    hs_1_26_MRI_ESM2_0     (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_MRI_ESM2_0     (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_MPI_ESM1_2_LR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_MPI_ESM1_2_LR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_KIOST_ESM      (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_KIOST_ESM      (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_IPSL_CM6A_LR   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_IPSL_CM6A_LR   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_CMCC_CM2_SR5   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_CMCC_CM2_SR5   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_AWI_CM_1_1_MR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_AWI_CM_1_1_MR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
Attributes: (12/36)
    AUTHOR:                        Commonwealth Scientific and Industrial Res...
    CITATION:                      CSIRO. (Year).  2071-2100 CMIP6 MRI-ESM2-0...
    COMMENT:                       The dataset includes significant wave heig...
    CRS:                           EPSG:4326
    Conventions:                   CF-1.8
    DESCRIPTION:                   The dataset contains projections of signif...
    ...                            ...
    northernmost_latitude:         90.
    product_name:                  ww3.207101_hs.nc
    southernmost_latitude:         -90.
    start_date:                    2071-01-01 00:00:00
    stop_date:                     2071-01-31 21:00:00
    westernmost_longitude:         0.

<IPython.core.display.Javascript object>

In [6]:
# combined_all = ds
metadata_json=orig_dataset_dir.joinpath("metadata_All_Phase2_2071-2100.json")

f_global = open(metadata_json)
meta_global = json.load(f_global)


for attr_name, attr_val in meta_global.items():
    if attr_name == 'PROVIDERS':
        attr_val = json.dumps(attr_val)
    combined_all_rechunk.attrs[attr_name] = attr_val

combined_all_rechunk.attrs['Conventions'] = "CF-1.8"
combined_all_rechunk

<xarray.Dataset>
Dimensions:                (latitude: 361, longitude: 720, time: 87656)
Coordinates:
  * latitude               (latitude) float32 -90.0 -89.5 -89.0 ... 89.5 90.0
  * longitude              (longitude) float32 0.0 0.5 1.0 ... 358.5 359.0 359.5
  * time                   (time) datetime64[ns] 2071-01-01 ... 2100-12-31T21...
Data variables:
    hs_1_26_MRI_ESM2_0     (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_MRI_ESM2_0     (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_MPI_ESM1_2_LR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_MPI_ESM1_2_LR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_KIOST_ESM      (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_KIOST_ESM      (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_IPSL_CM6A_LR   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_IPSL_CM6A_LR   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_CMCC_CM2_SR5   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_CMCC_CM2_SR5   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_AWI_CM_1_1_MR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_AWI_CM_1_1_MR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
Attributes: (12/36)
    AUTHOR:                        Commonwealth Scientific and Industrial Res...
    CITATION:                      CSIRO. (Year).  2071-2100 CMIP6  Phase 2 m...
    COMMENT:                       The dataset includes significant wave heig...
    CRS:                           EPSG:4326
    Conventions:                   CF-1.8
    DESCRIPTION:                   The dataset contains projections of signif...
    ...                            ...
    northernmost_latitude:         90.
    product_name:                  ww3.207101_hs.nc
    southernmost_latitude:         -90.
    start_date:                    2071-01-01 00:00:00
    stop_date:                     2071-01-31 21:00:00
    westernmost_longitude:         0.

<IPython.core.display.Javascript object>

In [7]:
# Resample the data annually and calculate the maximum value for each year
annual_maxima_combined_all_rechunk = combined_all_rechunk.resample(time='1Y').max(dim='time')

# Print the annual maxima
annual_maxima_combined_all_rechunk

<xarray.Dataset>
Dimensions:                (latitude: 361, longitude: 720, time: 30)
Coordinates:
  * latitude               (latitude) float32 -90.0 -89.5 -89.0 ... 89.5 90.0
  * longitude              (longitude) float32 0.0 0.5 1.0 ... 358.5 359.0 359.5
  * time                   (time) datetime64[ns] 2071-12-31 ... 2100-12-31
Data variables:
    hs_1_26_MRI_ESM2_0     (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_MRI_ESM2_0     (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_MPI_ESM1_2_LR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_MPI_ESM1_2_LR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_KIOST_ESM      (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_KIOST_ESM      (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_IPSL_CM6A_LR   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_IPSL_CM6A_LR   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_CMCC_CM2_SR5   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_CMCC_CM2_SR5   (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_5_85_AWI_CM_1_1_MR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
    hs_1_26_AWI_CM_1_1_MR  (time, latitude, longitude) float32 dask.array<chunksize=(1, 361, 720), meta=np.ndarray>
Attributes: (12/36)
    AUTHOR:                        Commonwealth Scientific and Industrial Res...
    CITATION:                      CSIRO. (Year).  2071-2100 CMIP6  Phase 2 m...
    COMMENT:                       The dataset includes significant wave heig...
    CRS:                           EPSG:4326
    Conventions:                   CF-1.8
    DESCRIPTION:                   The dataset contains projections of signif...
    ...                            ...
    northernmost_latitude:         90.
    product_name:                  ww3.207101_hs.nc
    southernmost_latitude:         -90.
    start_date:                    2071-01-01 00:00:00
    stop_date:                     2071-01-31 21:00:00
    westernmost_longitude:         0.

<IPython.core.display.Javascript object>

In [8]:
comb_dataset_dir_path = gca_data_dir.joinpath(r"Extreme_Wave_Height\GCM_All_CMIP6-phase2_Projection_2071-2100","Combined_CMIP_P2_WW3_3Hourly_2071-2100_ann_max.nc")

with ProgressBar():
    # combined_all_rechunk.to_zarr(str(comb_dataset_dir_path).replace(".nc", ".zarr"), mode="w")
    annual_maxima_combined_all_rechunk.to_zarr(str(comb_dataset_dir_path).replace(".nc", ".zarr"), mode="w")

[                                        ] | 0% Completed | 235.19 ss

In [ ]:
comb_dataset_dir_path = gca_data_dir.joinpath(r"Extreme_Wave_Height\GCM_All_CMIP6-phase2_Projection_2071-2100","Combined_CMIP_P2_WW3_3Hourly_2071-2100.nc")
dataset_dir_path_CF=pathlib.Path().joinpath(str(comb_dataset_dir_path)[0:-3]+"_CF"+".nc")
combined_all.isel(time=0).to_netcdf(path=dataset_dir_path_CF)

%%capture cap --no-stderr
# check original CF compliancy (for first file)

check_compliancy(testfile=dataset_dir_path_CF, 
                working_dir=CF_dir
                )
# save original CF compliancy (for first file)
save_compliancy(cap, testfile=dataset_dir_path_CF, working_dir=CF_dir)